Scraping from http://fbref.com

Data:
- outfield players stats
- goalkeepers stats
- team stats (of and against)

Original code by parth1902:
https://github.com/parth1902/Scrape-FBref-data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
#standard(stats)
stats = ["player","nationality","position","team","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]
stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"] 
#goalkeeping(keepers)
keepers = ["player","nationality","position","team","age","birth_year","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]
keepers3 = ["players_used","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]
#advance goalkeeping(keepersadv)
keepersadv = ["player","nationality","position","team","age","birth_year","minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]
keepersadv2 = ["minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]
#shooting(shooting)
shooting = ["player","nationality","position","team","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
#passing(passing)
passing = ["player","nationality","position","team","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
#passtypes(passing_types)
passing_types = ["player","nationality","position","team","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
#goal and shot creation(gca)
gca = ["player","nationality","position","team","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
#defensive actions(defense)
defense = ["player","nationality","position","team","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
#possession(possession)
possession = ["player","nationality","position","team","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
#playingtime(playingtime)
playingtime = ["player","nationality","position","team","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
#miscallaneous(misc)
misc = ["player","nationality","position","team","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]

def get_tables(url,text):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    res = requests.get(url, headers = headers)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    
    team_table = all_tables[0]
    team_vs_table = all_tables[1]
    player_table = all_tables[2]
    if text == 'for':
      return player_table, team_table
    if text == 'vs':
      return player_table, team_vs_table

def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                if(cell == None):
                  continue
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))                  
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player

def get_frame_team(features, team_table):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    for row in rows_squad:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"team"}).text.strip().encode().decode("utf-8")
            if 'team' in pre_df_squad:
                pre_df_squad['team'].append(name)
            else:
                pre_df_squad['team'] = [name]
            for f in features_wanted_squad:
                cell = row.find("td",{"data-stat": f})
                if(cell == None):
                  continue
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad

def get_frame_team_2(features, team_table):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    for row in rows_squad:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"squad"}).text.strip().encode().decode("utf-8")
            if 'squad' in pre_df_squad:
                pre_df_squad['squad'].append(name)
            else:
                pre_df_squad['squad'] = [name]
            for f in features_wanted_squad:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))
                    print(f)
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad

def frame_for_category(category,top,end,features):
    url = (top + category + end)
    player_table, team_table = get_tables(url,'for')
    df_player = get_frame(features, player_table)
    return df_player

def frame_for_category_team(category,top,end,features,text):
    url = (top + category + end)
    player_table, team_table = get_tables(url,text)
    df_team = get_frame_team(features, team_table)
    return df_team




#Function to get the player data for outfield player, includes all categories - standard stats, shooting
#passing, passing types, goal and shot creation, defensive actions, possession, and miscallaneous
def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end,stats)
    df2 = frame_for_category('shooting',top,end,shooting2)
    df3 = frame_for_category('passing',top,end,passing2)
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    df5 = frame_for_category('gca',top,end,gca2)
    df6 = frame_for_category('defense',top,end,defense2)
    df7 = frame_for_category('possession',top,end,possession2)
    df8 = frame_for_category('misc',top,end,misc2)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top,end):
    df1 = frame_for_category('keepers',top,end,keepers)
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df



#Function to get team-wise data accross all categories as mentioned above
def get_team_data(top,end,text):
    df1 = frame_for_category_team('stats',top,end,stats3,text)
    df2 = frame_for_category_team('keepers',top,end,keepers3,text)
    df3 = frame_for_category_team('keepersadv',top,end,keepersadv2,text)
    df4 = frame_for_category_team('shooting',top,end,shooting3,text)
    df5 = frame_for_category_team('passing',top,end,passing2,text)
    df6 = frame_for_category_team('passing_types',top,end,passing_types2,text)
    df7 = frame_for_category_team('gca',top,end,gca2,text)
    df8 = frame_for_category_team('defense',top,end,defense2,text)
    df9 = frame_for_category_team('possession',top,end,possession2,text)
    df10 = frame_for_category_team('misc',top,end,misc2,text)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df

In [3]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

df_outfield.to_csv('fbref_data/outfield_players.csv', index=False)

df_outfield

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,James Abankwah,ie IRL,DF,Udinese,19-109,2004,1.0,0.0,5.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Oliver Abildgaard,dk DEN,MF,Hellas Verona,26-329,1996,8.0,4.0,377.0,0.0,...,9.0,2.0,0.0,0.0,0.0,0.0,24.0,31.0,11.0,73.8
2,Tammy Abraham,eng ENG,FW,Roma,25-215,1997,33.0,24.0,2084.0,8.0,...,32.0,45.0,11.0,1.0,0.0,0.0,43.0,78.0,71.0,52.3
3,Christian Acella,it ITA,MF,Cremonese,20-302,2002,1.0,0.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Francesco Acerbi,it ITA,DF,Inter,35-084,1988,26.0,22.0,2095.0,0.0,...,14.0,10.0,1.0,0.0,0.0,0.0,131.0,71.0,35.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,Petar Zovko,ba BIH,GK,Spezia,21-041,2002,1.0,0.0,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
577,Szymon Żurkowski,pl POL,MF,Fiorentina,25-222,1997,2.0,0.0,32.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,50.0
578,Szymon Żurkowski,pl POL,MF,Spezia,25-222,1997,6.0,2.0,240.0,0.0,...,8.0,5.0,0.0,0.0,0.0,0.0,18.0,3.0,2.0,60.0
579,Milan Đurić,ba BIH,FW,Hellas Verona,32-348,1990,23.0,9.0,974.0,1.0,...,22.0,20.0,4.0,0.0,0.0,0.0,23.0,146.0,38.0,79.3


In [4]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

df_keeper.to_csv('fbref_data/keepers_players.csv', index=False)

df_keeper

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Emil Audero,it ITA,GK,Sampdoria,26-107,1997,25.0,25.0,2250.0,39.0,...,39.1,200.0,57.0,44.2,356.0,24.0,6.7,23.0,0.92,13.9
1,Francesco Bardi,it ITA,GK,Bologna,31-107,1992,1.0,1.0,90.0,0.0,...,35.6,9.0,44.4,33.4,10.0,0.0,0.0,0.0,0.00,0.0
2,Marco Carnesecchi,it ITA,GK,Cremonese,22-308,2000,24.0,24.0,2160.0,40.0,...,41.4,179.0,65.9,52.8,351.0,28.0,8.0,24.0,1.00,13.9
3,Michele Cerofolini,it ITA,GK,Fiorentina,24-121,1999,1.0,1.0,90.0,0.0,...,31.1,3.0,100.0,56.3,2.0,0.0,0.0,0.0,0.00,5.0
4,Andrea Consigli,it ITA,GK,Sassuolo,36-098,1987,31.0,31.0,2790.0,46.0,...,33.8,265.0,31.3,32.8,424.0,26.0,6.1,26.0,0.84,13.9
5,Alessio Cragno,it ITA,GK,Monza,28-311,1994,1.0,1.0,90.0,3.0,...,22.6,6.0,16.7,33.3,7.0,0.0,0.0,2.0,2.00,25.7
6,Michele Di Gregorio,it ITA,GK,Monza,25-282,1997,32.0,32.0,2880.0,41.0,...,32.6,209.0,37.3,34.7,469.0,13.0,2.8,27.0,0.84,13.3
7,Bartłomiej Drągowski,pl POL,GK,Spezia,25-259,1997,30.0,30.0,2586.0,48.0,...,34.1,179.0,60.9,47.3,428.0,13.0,3.0,32.0,1.11,14.2
8,Wladimiro Falcone,it ITA,GK,Lecce,28-023,1995,33.0,33.0,2970.0,40.0,...,41.7,239.0,76.6,52.3,470.0,22.0,4.7,37.0,1.12,14.1
9,Pierluigi Gollini,it ITA,GK,Fiorentina,28-048,1995,3.0,3.0,270.0,2.0,...,32.4,11.0,63.6,49.3,24.0,1.0,4.2,0.0,0.00,9.3


In [5]:
df_team = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats', 'for')

df_team.to_csv('fbref_data/teams.csv', index=False)

df_team

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Atalanta,25.0,49.8,33.0,363.0,2970.0,55.0,35.0,6.0,8.0,...,399.0,335.0,49.0,6.0,3.0,2.0,1968.0,508.0,418.0,54.9
1,Bologna,27.0,54.0,33.0,363.0,2970.0,41.0,32.0,6.0,6.0,...,412.0,378.0,69.0,5.0,9.0,3.0,1836.0,325.0,397.0,45.0
2,Cremonese,32.0,43.0,33.0,363.0,2970.0,29.0,14.0,4.0,6.0,...,415.0,349.0,64.0,4.0,5.0,1.0,1817.0,500.0,660.0,43.1
3,Empoli,31.0,46.7,33.0,363.0,2970.0,28.0,15.0,1.0,1.0,...,379.0,390.0,43.0,1.0,6.0,1.0,1557.0,343.0,424.0,44.7
4,Fiorentina,30.0,56.8,33.0,363.0,2970.0,43.0,32.0,4.0,6.0,...,421.0,461.0,45.0,3.0,5.0,2.0,1798.0,539.0,458.0,54.1
5,Hellas Verona,36.0,41.7,33.0,363.0,2970.0,25.0,19.0,1.0,1.0,...,476.0,337.0,65.0,1.0,1.0,2.0,1954.0,666.0,650.0,50.6
6,Inter,24.0,56.9,33.0,363.0,2970.0,58.0,41.0,4.0,5.0,...,390.0,360.0,61.0,4.0,3.0,2.0,1723.0,465.0,369.0,55.8
7,Juventus,29.0,48.7,33.0,363.0,2970.0,50.0,37.0,3.0,6.0,...,383.0,354.0,57.0,6.0,4.0,0.0,1685.0,425.0,408.0,51.0
8,Lazio,22.0,51.3,33.0,363.0,2970.0,51.0,32.0,5.0,7.0,...,347.0,421.0,46.0,5.0,1.0,0.0,1734.0,334.0,333.0,50.1
9,Lecce,29.0,41.7,33.0,363.0,2970.0,26.0,17.0,3.0,4.0,...,483.0,383.0,63.0,3.0,5.0,3.0,1843.0,499.0,624.0,44.4


In [6]:
df_vsteam = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats', 'vs')

df_vsteam.to_csv('fbref_data/teams_vs.csv', index=False)

df_vsteam

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Atalanta,25.0,50.2,33.0,363.0,2970.0,37.0,28.0,3.0,3.0,...,354.0,372.0,42.0,1.0,8.0,1.0,1976.0,418.0,508.0,45.1
1,vs Bologna,27.0,46.0,33.0,363.0,2970.0,40.0,24.0,7.0,9.0,...,400.0,388.0,54.0,6.0,6.0,1.0,1775.0,397.0,325.0,55.0
2,vs Cremonese,32.0,57.0,33.0,363.0,2970.0,58.0,39.0,5.0,5.0,...,373.0,393.0,57.0,4.0,6.0,0.0,1867.0,660.0,500.0,56.9
3,vs Empoli,31.0,53.3,33.0,363.0,2970.0,42.0,28.0,4.0,6.0,...,412.0,365.0,67.0,4.0,1.0,1.0,1693.0,424.0,343.0,55.3
4,vs Fiorentina,30.0,43.2,33.0,363.0,2970.0,37.0,26.0,5.0,5.0,...,487.0,400.0,80.0,3.0,6.0,2.0,1708.0,458.0,539.0,45.9
5,vs Hellas Verona,36.0,58.3,33.0,363.0,2970.0,49.0,43.0,0.0,1.0,...,358.0,455.0,35.0,1.0,1.0,2.0,1833.0,650.0,666.0,49.4
6,vs Inter,24.0,43.1,33.0,363.0,2970.0,33.0,28.0,3.0,3.0,...,386.0,359.0,31.0,3.0,5.0,2.0,1532.0,369.0,465.0,44.2
7,vs Juventus,29.0,51.3,33.0,363.0,2970.0,28.0,22.0,3.0,4.0,...,370.0,345.0,47.0,1.0,6.0,0.0,1662.0,408.0,425.0,49.0
8,vs Lazio,22.0,48.7,33.0,363.0,2970.0,24.0,16.0,1.0,1.0,...,447.0,330.0,58.0,1.0,7.0,1.0,1788.0,333.0,334.0,49.9
9,vs Lecce,29.0,58.3,33.0,363.0,2970.0,37.0,25.0,5.0,5.0,...,401.0,455.0,61.0,4.0,4.0,2.0,1782.0,624.0,499.0,55.6


In [7]:
print('Outfield player columns')
print(df_outfield.columns.values)

print('Keeper player columns')
print(df_keeper.columns.values)

print('Team columns')
print(df_team.columns.values)

print('Vs Team columns')
print(df_team.columns.values)

Outfield player columns
['player' 'nationality' 'position' 'team' 'age' 'birth_year' 'games'
 'games_starts' 'minutes' 'goals' 'assists' 'pens_made' 'pens_att'
 'cards_yellow' 'cards_red' 'goals_per90' 'assists_per90'
 'goals_assists_per90' 'goals_pens_per90' 'goals_assists_pens_per90' 'xg'
 'npxg' 'xg_per90' 'npxg_per90' 'minutes_90s' 'shots_on_target'
 'shots_free_kicks' 'shots_on_target_pct' 'shots_on_target_per90'
 'goals_per_shot' 'goals_per_shot_on_target' 'npxg_per_shot' 'xg_net'
 'npxg_net' 'passes_completed' 'passes' 'passes_pct'
 'passes_total_distance' 'passes_progressive_distance'
 'passes_completed_short' 'passes_short' 'passes_pct_short'
 'passes_completed_medium' 'passes_medium' 'passes_pct_medium'
 'passes_completed_long' 'passes_long' 'passes_pct_long' 'assisted_shots'
 'passes_into_final_third' 'passes_into_penalty_area'
 'crosses_into_penalty_area' 'progressive_passes' 'passes_live'
 'passes_dead' 'passes_free_kicks' 'through_balls' 'passes_switches'
 'crosses' 'corn

Scrape data for past seasons. 

Do not repeat if data is already present.

In [8]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats')

df_outfield.to_csv('fbref_data/season2122/outfield_players.csv', index=False)

df_outfield

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Tammy Abraham,eng ENG,FW,Roma,23,1997,37.0,36.0,3084.0,17.0,...,38.0,49.0,17.0,1.0,1.0,0.0,68.0,78.0,80.0,49.4
1,Francesco Acerbi,it ITA,DF,Lazio,33,1988,30.0,29.0,2536.0,4.0,...,18.0,16.0,4.0,1.0,0.0,0.0,162.0,69.0,43.0,61.6
2,Michel Aebischer,ch SUI,MF,Bologna,24,1997,12.0,4.0,443.0,0.0,...,13.0,4.0,0.0,0.0,0.0,0.0,27.0,5.0,4.0,55.6
3,Felix Afena-Gyan,gh GHA,"FW,MF",Roma,18,2003,17.0,6.0,668.0,2.0,...,15.0,24.0,4.0,0.0,0.0,0.0,31.0,11.0,15.0,42.3
4,Kevin Agudelo,co COL,"MF,FW",Spezia,22,1998,23.0,12.0,1237.0,3.0,...,36.0,24.0,0.0,0.0,0.0,0.0,67.0,15.0,14.0,51.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,Nadir Zortea,it ITA,"DF,MF",Salernitana,22,1999,29.0,13.0,1410.0,1.0,...,14.0,6.0,0.0,0.0,0.0,0.0,72.0,8.0,17.0,32.0
628,Petar Zovko,ba BIH,GK,Spezia,19,2002,1.0,0.0,29.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
629,Szymon Żurkowski,pl POL,MF,Empoli,23,1997,35.0,29.0,2307.0,6.0,...,41.0,67.0,0.0,0.0,1.0,0.0,162.0,24.0,32.0,42.9
630,Milan Đurić,ba BIH,FW,Salernitana,31,1990,33.0,23.0,2165.0,5.0,...,24.0,45.0,6.0,1.0,0.0,0.0,40.0,242.0,83.0,74.5


In [ ]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats')

df_keeper.to_csv('fbref_data/season2122/keepers_players.csv', index=False)

df_keeper

In [ ]:
df_team = get_team_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats', 'for')

df_team.to_csv('fbref_data/season2122/teams.csv', index=False)

df_team

In [ ]:
df_vsteam = get_team_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats', 'vs')

df_vsteam.to_csv('fbref_data/season2122/teams_vs.csv', index=False)

df_vsteam

In [ ]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats')
df_outfield.to_csv('fbref_data/season2021/outfield_players.csv', index=False)

df_keeper = get_keeper_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats')
df_keeper.to_csv('fbref_data/season2021/keepers_players.csv', index=False)

df_team = get_team_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats', 'for')
df_team.to_csv('fbref_data/season2021/teams.csv', index=False)

df_vsteam = get_team_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats', 'vs')
df_vsteam.to_csv('fbref_data/season2021/teams_vs.csv', index=False)
